<a href="https://colab.research.google.com/github/canasep/mackprojeto2/blob/main/codes/PROJETO_APLICADO_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*NESTA ETAPA IREMOS CLONAR O GITHUB NO COLLAB*

In [1]:
!git clone https://github.com/canasep/mackprojeto2.git

Cloning into 'mackprojeto2'...
remote: Enumerating objects: 54, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 54 (delta 14), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (54/54), 3.83 MiB | 11.41 MiB/s, done.
Resolving deltas: 100% (14/14), done.


**CRIAÇÃO DO DATASET A PARTIR DO CSV CRÚ**

In [2]:
# IMPORTAÇÃO DO CSV E CRIAÇÃO DO DATASET

## IMPORTANDO PACOTE PANDAS

import pandas as pd

## CAMINHO DO ARQUIVO

file_path = '/content/mackprojeto2/dataset/arquivos/amazon_products_sales_data_uncleaned.csv'

## CÓDIGO DE IMPORTAÇÃO E CRIAÇÃO DO DATASET DF

try:
    df = pd.read_csv(file_path)
    print("Dataset criado com sucesso!")
except Exception as e:
    print(f"Erro ao criar o dataset: {e}")


Dataset criado com sucesso!


**IMPRESSÃO DO HEAD DO DATASET DF**

In [5]:
# IMPRESSÃO DO HEAD DO DATASET DF

df.head()


,title,rating,number_of_reviews,bought_in_last_month,current/discounted_price,price_on_variant,listed_price,is_best_seller,is_sponsored,is_couponed,buy_box_availability,delivery_details,sustainability_badges,image_url,product_url,collected_at
0,BOYA BOYALINK 2 Wireless Lavalier Microphone f...,4.6 out of 5 stars,375,300+ bought in past month,89.68,basic variant price: 2.4GHz,$159.00,No Badge,Sponsored,Save 15% with coupon,Add to cart,"Delivery Mon, Sep 1",Carbon impact,https://m.media-amazon.com/images/I/71pAqiVEs3...,/sspa/click?ie=UTF8&spc=MTo4NzEzNDY2NTQ5NDYxND...,2025-08-21 11:14:29
1,"LISEN USB C to Lightning Cable, 240W 4 in 1 Ch...",4.3 out of 5 stars,"2,457",6K+ bought in past month,9.99,basic variant price: nan,$15.99,No Badge,Sponsored,No Coupon,Add to cart,"Delivery Fri, Aug 29",NaN,https://m.media-amazon.com/images/I/61nbF6aVIP...,/sspa/click?ie=UTF8&spc=MTo4NzEzNDY2NTQ5NDYxND...,2025-08-21 11:14:29
2,"DJI Mic 2 (2 TX + 1 RX + Charging Case), Wirel...",4.6 out of 5 stars,"3,044",2K+ bought in past month,314.00,basic variant price: nan,$349.00,No Badge,Sponsored,No Coupon,Add to cart,"Delivery Mon, Sep 1",NaN,https://m.media-amazon.com/images/I/61h78MEXoj...,/sspa/click?ie=UTF8&spc=MTo4NzEzNDY2NTQ5NDYxND...,2025-08-21 11:14:29
3,"Apple AirPods Pro 2 Wireless Earbuds, Active N...",4.6 out of 5 stars,"35,882",10K+ bought in past month,NaN,basic variant price: $162.24,No Discount,Best Seller,Organic,No Coupon,NaN,NaN,NaN,https://m.media-amazon.com/images/I/61SUj2aKoE...,/Apple-Cancellation-Transparency-Personalized-...,2025-08-21 11:14:29
4,Apple AirTag 4 Pack. Keep Track of and find Yo...,4.8 out of 5 stars,"28,988",10K+ bought in past month,NaN,basic variant price: $72.74,No Discount,No Badge,Organic,No Coupon,NaN,NaN,NaN,https://m.media-amazon.com/images/I/61bMNCeAUA...,/Apple-MX542LL-A-AirTag-Pack/dp/B0D54JZTHY/ref...,2025-08-21 11:14:29
